In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['TAVILY_API_KEY']= os.getenv('TAVILY_API_KEY')

### [To-do] 웨딩 정보 관련 데이터 수집 -> 코드 완성

In [10]:
from typing import Dict, Optional

from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

class PDFBasedRAG:
    def __init__(
        self, 
        model_name: str = "gpt-4o-mini",
        temperature: float = 0.1,
        chunk_size: int = 1000,
        chunk_overlap: int = 200
    ):
      
        self.llm = ChatOpenAI(
            model_name=model_name,
            temperature=temperature
        )
        
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )
        
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len
        )
        
        self.vector_store = None
        self.qa_chain = None

    def load_pdf(self, pdf_path: str) -> None:
        """
        PDF 파일 로드 및 처리
        
        Args:
            pdf_path: PDF 파일 경로
        """
        
        if not os.path.exists(pdf_path):
            raise FileNotFoundError(f"PDF 파일을 찾을 수 없습니다: {pdf_path}")
  
        loader = TextLoader(pdf_path, encoding='utf-8')
        documents = loader.load()

        split_documents = self.text_splitter.split_documents(documents)
        
        # 벡터 저장소 생성
        self.vector_store = FAISS.from_documents(
            documents=split_documents,
            embedding=self.embeddings
        )
        
        # QA 체인 설정
        self._setup_qa_chain()
        
        print(f"PDF 로드 완료: 총 {len(split_documents)}개의 문서 청크 생성")


    def _setup_qa_chain(self) -> None:
        """QA 체인 설정"""
        # 프롬프트 템플릿 
        prompt_template = """주어진 컨텍스트를 기반으로 질문에 답변해주세요.
        
        컨텍스트에서 답을 찾을 수 없는 경우:
        1) "제공된 PDF에서 관련 정보를 찾을 수 없습니다."라고 명시하고
        2) 일반적인 지식을 바탕으로 답변해주세요.

        컨텍스트:
        {context}

        질문: {question}

        """

        PROMPT = PromptTemplate(
            template=prompt_template,
            input_variables=["context", "question"]
        )

        # QA 체인 생성
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.vector_store.as_retriever(
                search_kwargs={"k": 3}
            ),
            return_source_documents=True,
            chain_type_kwargs={"prompt": PROMPT}
        )

    def answer_query(self, query: str) -> Dict:
        """
        질문에 대한 답변 생성
        
        Args:
            query: 질문 내용
            
        Returns:
            답변과 참조 문서 정보를 포함한 딕셔너리
        """

        if self.vector_store is None:
            # PDF가 로드되지 않은 경우 LLM만 사용
            response = self.llm.predict(query)
            return {
                "answer": "PDF가 로드되지 않아 일반적인 답변만 제공됩니다:\n" + response,
                "source_documents": []
            }

        # RAG를 사용한 답변 생성
        response = self.qa_chain({"query": query})
        
        return {
            "answer": response["result"],
            "source_documents": [
                {
                    "content": doc.page_content,
                    "page": doc.metadata.get("page", "Unknown")
                }
                for doc in response["source_documents"]
            ]
        }

def main():
   
    try:
        rag = PDFBasedRAG(
            temperature=0.1
        )
        
        pdf_path = "wedding_info_data.txt"  #  파일 경로
        rag.load_pdf(pdf_path)
        

        while True:
            user_input = input("\n질문: ")
            
            if user_input.lower() == 'quit':
                print("상담을 종료합니다. 감사합니다!")
                break
                
            try:
                response = rag.answer_query(user_input)
                print("\n답변:", response["answer"])
                print("\n참조된 페이지:")
                for source in response["source_documents"]:
                    print(f"- Page {source['page']}")
            except Exception as e:
                print(f"\n오류가 발생했습니다: {str(e)}")
        
                
    except Exception as e:
        print(f"에러 발생: {str(e)}")

if __name__ == "__main__":
    main()

PDF 로드 완료: 총 4개의 문서 청크 생성


/var/folders/k3/f4f9x_vx7zg0vcl_gmq4gd0c0000gn/T/ipykernel_72171/789810345.py:118: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = self.qa_chain({"query": query})



답변: 제공된 PDF에서 관련 정보를 찾을 수 없습니다. 

일반적인 지식을 바탕으로 답변드리자면, 예산 관리를 위해 다음과 같은 팁을 고려할 수 있습니다:

1. **예산 설정**: 전체 예산을 설정하고, 각 항목(예: 드레스, 메이크업, 촬영, 음식 등)에 대한 예산을 분배합니다.

2. **우선순위 정하기**: 가장 중요하게 생각하는 항목에 더 많은 예산을 할당하고, 덜 중요한 항목은 줄이는 방법을 고려합니다.

3. **비교 쇼핑**: 여러 업체의 가격을 비교하고, 리뷰를 참고하여 가성비가 좋은 선택을 합니다.

4. **비용 절감 방법**: DIY(Do It Yourself)로 할 수 있는 부분은 직접 하거나, 친구나 가족의 도움을 받는 것도 좋은 방법입니다.

5. **예비비 마련**: 예상치 못한 비용이 발생할 수 있으므로, 전체 예산의 10% 정도는 예비비로 남겨두는 것이 좋습니다.

6. **계약서 확인**: 계약서를 꼼꼼히 읽고, 숨겨진 비용이 없는지 확인합니다.

7. **할인 및 프로모션 활용**: 결혼식 관련 업체에서 제공하는 할인이나 프로모션을 적극 활용합니다.

이러한 팁들을 통해 예산을 효과적으로 관리할 수 있습니다.

참조된 페이지:
- Page Unknown
- Page Unknown
- Page Unknown
상담을 종료합니다. 감사합니다!
